<a href="https://colab.research.google.com/github/chitransh0052/FLAM_ASSIGNMENT/blob/master/FLAM_ASSIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python rembg mediapipe torch torchvision numpy matplotlib pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from rembg import remove
import mediapipe as mp
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms




 **Task 1: Capturing and Preparing the Person's Image**



In [ ]:
def extract_foreground(image_path):
    with open(image_path, 'rb') as i:
        input_data = i.read()
    output = remove(input_data)
    with open("person_no_bg.png", 'wb') as o:
        o.write(output)

extract_foreground("person.jpg")


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 188GB/s]


**Task 2: Analyzing Shadows and Lighting of the Background Image**

In [ ]:
def detect_shadows(bg_img_path):
    img = cv2.imread(bg_img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Shadow mask using threshold
    _, shadow_mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # Classify shadows
    blurred = cv2.GaussianBlur(gray, (21, 21), 0)
    hard_shadows = cv2.absdiff(gray, blurred)
    _, hard_mask = cv2.threshold(hard_shadows, 30, 255, cv2.THRESH_BINARY)

    soft_mask = cv2.bitwise_and(shadow_mask, cv2.bitwise_not(hard_mask))
    return hard_mask, soft_mask

hard, soft = detect_shadows("scene.jpeg")
cv2.imwrite("hard_shadow_mask.png", hard)
cv2.imwrite("soft_shadow_mask.png", soft)


True

**Task 3: Determining Light Direction**

In [ ]:
def estimate_light_direction(shadow_mask):
    contours, _ = cv2.findContours(shadow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None

    largest = max(contours, key=cv2.contourArea)
    [vx, vy, x, y] = cv2.fitLine(largest, cv2.DIST_L2, 0, 0.01, 0.01)
    return (vx[0], vy[0])  # light vector direction

light_dir = estimate_light_direction(hard)
print("Estimated Light Direction Vector:", light_dir)


Estimated Light Direction Vector: (0.6099807, 0.7924163)


**Task 4: Coloring and Blending**

In [ ]:
def match_histogram(source, reference):
    source_lab = cv2.cvtColor(source, cv2.COLOR_RGB2LAB)
    ref_lab = cv2.cvtColor(reference, cv2.COLOR_RGB2LAB)

    for i in range(3):
        s_mean, s_std = source_lab[:,:,i].mean(), source_lab[:,:,i].std()
        r_mean, r_std = ref_lab[:,:,i].mean(), ref_lab[:,:,i].std()

        source_lab[:,:,i] = (source_lab[:,:,i] - s_mean) * (r_std / s_std) + r_mean
        source_lab[:,:,i] = np.clip(source_lab[:,:,i], 0, 255)

    return cv2.cvtColor(source_lab.astype("uint8"), cv2.COLOR_LAB2RGB)


**Task 5: Generating Final Output**

In [ ]:
def integrate_person_to_scene(scene_path, person_path):
    scene = cv2.imread(scene_path)
    scene = cv2.cvtColor(scene, cv2.COLOR_BGR2RGB)

    person = Image.open(person_path).convert("RGBA")
    person = person.resize((300, 600))  # Resize as needed

    # Color match
    person_np = np.array(person)[...,:3]
    matched = match_histogram(person_np, scene)

    # Create alpha blended image
    alpha = np.array(person)[...,3] / 255.0
    y_offset = scene.shape[0] - matched.shape[0]
    x_offset = 50

    for c in range(3):
        scene[y_offset:y_offset+matched.shape[0], x_offset:x_offset+matched.shape[1], c] = (
            alpha * matched[:, :, c] +
            (1 - alpha) * scene[y_offset:y_offset+matched.shape[0], x_offset:x_offset+matched.shape[1], c]
        )

    final = cv2.cvtColor(scene, cv2.COLOR_RGB2BGR)
    cv2.imwrite("final_composite.jpg", final)

integrate_person_to_scene("scene.jpeg", "person_no_bg.png")
